https://www.rankedchoicevoting.org/data_clearinghouse
    
https://drive.google.com/drive/folders/1DJzIrTaDW3GSGJTkPTGAlpAMbozFG_pm
    
Objective for this notebook: separate the elections into the following categories:
* Leading candidate in the first round has greater than 50% first choice votes
* Leading candidate in the first round has between 45-50% first choice votes
* Leading candidate in the first round has less than 45% of first choice votes

In [1]:
import glob
import pandas
print('pandas',pandas.__version__)

pandas 0.23.4


# import data

In [2]:
list_of_files = glob.glob('voting_data/San_Fransisco/San Francisco/**/*')
len(list_of_files)

46

In [3]:
list_of_files[0]

'voting_data/San_Fransisco/San Francisco/San Fran_Nov 2010_District 10 Supervisors/BallotImage-D10.txt'

not all the files are relevant. Need to develop a filter to identify relevant files

In [4]:
list_of_file_extentions=[]
for filename in list_of_files:
    #print(filename.split('.')[-1])
    list_of_file_extentions.append(filename.split('.')[-1])
print(set(list_of_file_extentions))

{'txt', 'pdf'}


In [5]:
for filename in list_of_files:
    if filename.endswith('.txt'):
#        print(filename)
        if 'ballot' in filename.lower():
            print(filename.split('/')[-1])
            with open(filename,'r') as fil:
                file_contents = fil.readlines()
            print(file_contents[0:2])
            print(len(file_contents[0].strip()))

BallotImage-D10.txt
['000000600001706700000040020000274001000014900\n', '000000600001706700000040020000274002000015700\n']
45
BallotImage-D2.txt
['000000700001712400000090020000331001000012600\n', '000000700001712400000090020000331002000000001\n']
45
BallotImageListing.txt
['\x1bE\x1b&l2a0o7c067F\x1b(s0p16.66h3b6T\x1b&a00L\n', 'BALLOT IMAGE LISTING                               SAN FRANCISCO                                      OFFICIAL RESULTS\n']
36
BallotImageSummary.txt
['\x1bE\x1b&l2a0o7c067F\x1b(s0p16.66h3b6T\x1b&a00L\n', 'BALLOT IMAGE ELECTION SUMMARY\n']
36
20151119_ballotimage.txt
['000000100002610500000010020000012001000003600\n', '000000100002610500000010020000012002000003700\n']
45
D10_BallotImage.txt
['000003300000385300000010020000054001000012900\n', '000003300000385300000010020000054002000012800\n']
45
Sheriff-BallotImage.txt
['000000200004728200000010020000003001000002800\n', '000000200004728200000010020000003002000002800\n']
45
DA-BallotImage.txt
['00000010000472820000

In [6]:
list_of_ballot_files=[]
for filename in list_of_files:
    if filename.endswith('.txt'):
        with open(filename,'r') as fil:
            file_contents = fil.readlines()
        if len(file_contents[0].strip())==45:
            print(filename)
            list_of_ballot_files.append(filename)
            print(file_contents[1])

voting_data/San_Fransisco/San Francisco/San Fran_Nov 2010_District 10 Supervisors/BallotImage-D10.txt
000000600001706700000040020000274002000015700

voting_data/San_Fransisco/San Francisco/San Fran_Nov 2010_District 2 Supervisors/BallotImage-D2.txt
000000700001712400000090020000331002000000001

voting_data/San_Fransisco/San Francisco/2015 All offices/20151119_ballotimage.txt
000000100002610500000010020000012002000003700

voting_data/San_Fransisco/San Francisco/San Fran_Nov 2014_District 10 Supervisors/D10_BallotImage.txt
000003300000385300000010020000054002000012800

voting_data/San_Fransisco/San Francisco/San Fran Nov 2011 Sheriff/Sheriff-BallotImage.txt
000000200004728200000010020000003002000002800

voting_data/San_Fransisco/San Francisco/San Fran Nov 2011 District Attorney/DA-BallotImage.txt
000000100004728200000010020000003002000002300

voting_data/San_Fransisco/San Francisco/San Fran_Nov 2010_District 8 Supervisors/BallotImage-D8.txt
000001000001706800000040020000513002000014200



In [8]:
reslts={'leading candidate in first round has more than 50% of first choice votes':[],
        'leading candidate in first round vote has between 50% and 45% of first choice votes':[],
        'leading candidate in first round vote has less than 45% of first choice votes':[]}

for ballot in list_of_ballot_files:
    #print(ballot)
    df = pandas.read_fwf(ballot,
                     header=None,
                     widths=[7,9,7,3,7,3,7,1,1])
    df.columns=['contest_id','pref_voter_id',
            'serial_number','tally_type_id',
            'precinct_id','vote_rank',
            'candidate_id','over_vote','under_vote']
    #print(df.shape)
    #print(df['candidate_id'].unique())
    df_cand = df[df['candidate_id']!=0] # drop rows where no candidate is specified
    series_of_candidates_and_first_choice_count = df_cand[df_cand['vote_rank']==1].groupby('candidate_id')['vote_rank'].count()
    number_of_first_choice_votes =  series_of_candidates_and_first_choice_count.sum()
    #print('number of first choice votes:',number_of_first_choice_votes)
    if (series_of_candidates_and_first_choice_count > number_of_first_choice_votes*0.5).any():
        #print('Leading candidate in the first round has greater than 50% first choice votes')
        reslts['leading candidate in first round has more than 50% of first choice votes'].append(ballot)
    elif ((series_of_candidates_and_first_choice_count <= number_of_first_choice_votes*0.5).any() and 
          (series_of_candidates_and_first_choice_count >= number_of_first_choice_votes*0.45).any()):
        #print('Leading candidate in the first round has between 45-50% first choice votes')
        reslts['leading candidate in first round vote has between 50% and 45% of first choice votes'].append(ballot)
    elif (series_of_candidates_and_first_choice_count < number_of_first_choice_votes*0.45).any():
        #print('Leading candidate in the first round has less than 45% of first choice votes')
        reslts['leading candidate in first round vote has less than 45% of first choice votes'].append(ballot)
    else:
        raise Exception('should not reach this condition')

voting_data/San_Fransisco/San Francisco/San Fran_Nov 2010_District 10 Supervisors/BallotImage-D10.txt
(61650, 9)
[149 157 163 147 165 160 153 159   0 158 148 156 162 152 161 155 151 166
 154 150 164 146  26]
number of first choice votes: 17705
voting_data/San_Fransisco/San Francisco/San Fran_Nov 2010_District 2 Supervisors/BallotImage-D2.txt
(86733, 9)
[126   0 120 121 124 122 123  25]
number of first choice votes: 24030
voting_data/San_Fransisco/San Francisco/2015 All offices/20151119_ballotimage.txt
(1271076, 9)
[36 37 39  0 38 40 41 63 65 62 66 61 64 42 43 44 47 45 46]
number of first choice votes: 390615
voting_data/San_Fransisco/San Francisco/San Fran_Nov 2014_District 10 Supervisors/D10_BallotImage.txt
(49026, 9)
[129 128 127 125 126   0]
number of first choice votes: 15312
voting_data/San_Fransisco/San Francisco/San Fran Nov 2011 Sheriff/Sheriff-BallotImage.txt
(591726, 9)
[28 27  0 26 25]
number of first choice votes: 182519
voting_data/San_Fransisco/San Francisco/San Fran Nov 

In [10]:
for k,v in reslts.items():
    print(k,':')
    for election in v:
        print('   ',election.replace('voting_data/','').replace('.txt',''))

leading candidate in first round has more than 50% of first choice votes :
leading candidate in first round vote has between 50% and 45% of first choice votes :
    San_Fransisco/San Francisco/San Fran_Nov 2014_District 10 Supervisors/D10_BallotImage
leading candidate in first round vote has less than 45% of first choice votes :
    San_Fransisco/San Francisco/San Fran_Nov 2010_District 10 Supervisors/BallotImage-D10
    San_Fransisco/San Francisco/San Fran_Nov 2010_District 2 Supervisors/BallotImage-D2
    San_Fransisco/San Francisco/2015 All offices/20151119_ballotimage
    San_Fransisco/San Francisco/San Fran Nov 2011 Sheriff/Sheriff-BallotImage
    San_Fransisco/San Francisco/San Fran Nov 2011 District Attorney/DA-BallotImage
    San_Fransisco/San Francisco/San Fran_Nov 2010_District 8 Supervisors/BallotImage-D8
    San_Fransisco/San Francisco/San Fran Nov 2011 Mayor/Mayor-BallotImage
    San_Fransisco/San Francisco/San Fran_Nov 2012_District 7 Supervisors/D7-BallotImage
    San_Fr